In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import pathlib 

2.9.2


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
data_dir = "/content/drive/MyDrive/MACHINE LEARNING PROJECT " 
data_dir = pathlib.Path(data_dir)
data_dir_test = pathlib.Path("/content/drive/MyDrive/MACHINE LEARNING PROJECT ")

In [ ]:
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

232


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 232 files belonging to 26 classes.
Using 186 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 232 files belonging to 26 classes.
Using 46 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 26

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
   tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=6
)

Epoch 1/6
6/6 [==============================] - 15s 2s/step - loss: 3.2660 - accuracy: 0.0430 - val_loss: 3.2859 - val_accuracy: 0.0435
Epoch 2/6
6/6 [==============================] - 7s 1s/step - loss: 3.2364 - accuracy: 0.0484 - val_loss: 3.3063 - val_accuracy: 0.0435
Epoch 3/6
6/6 [==============================] - 7s 1s/step - loss: 3.2179 - accuracy: 0.0430 - val_loss: 3.4152 - val_accuracy: 0.0000e+00
Epoch 4/6
6/6 [==============================] - 7s 1s/step - loss: 3.1998 - accuracy: 0.0591 - val_loss: 3.4155 - val_accuracy: 0.0000e+00
Epoch 5/6
6/6 [==============================] - 7s 1s/step - loss: 3.1797 - accuracy: 0.0699 - val_loss: 3.4285 - val_accuracy: 0.0000e+00
Epoch 6/6
6/6 [==============================] - 7s 1s/step - loss: 3.1556 - accuracy: 0.0860 - val_loss: 3.4568 - val_accuracy: 0.0000e+00


In [ ]:
def new_image(test_img_dir):
  data_dir = pathlib.Path(test_img_dir)
  img = tf.keras.utils.load_img(
    data_dir, target_size=(img_height, img_width))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  return img_array
holy_image = new_image("/content/drive/MyDrive/MACHINE LEARNING PROJECT /N/N10.png")

In [ ]:
pred = model.predict(holy_image)
score = tf.nn.softmax(pred)

1/1 [==============================] - 0s 174ms/step


In [ ]:
100 * np.max(score)
class_names[np.argmax(score)]

'H'

In [ ]:
model.save('my_model.h5')